In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scipy.io
import scanpy.external as scex
import sklearn.metrics
import seaborn as sns
from matplotlib import pyplot as plt
import scanpyHelpers as scH

In [ ]:
#adata=sc.read(results_file_post)
#adata.uns['log1p'] = {"base":None}

Set up out properties

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

In [ ]:
%matplotlib inline

Set scanpy out-files

In [ ]:
writeDir = "write/"

fileName = "LUADpdx"

resultsFile = writeDir + fileName + '.h5ad'       # final output
resultsFileQC = writeDir + fileName + '_QC.h5ad'  # post QC (pre-analysis) 

Set figure parameters

In [ ]:
sc.set_figure_params(scanpy=True, dpi=100, dpi_save=150, fontsize=10, format='png')
sc.settings.figdir = "figures/" + fileName + "/"
figName = fileName

In [ ]:
resultsFileQC = writeDir + fileName + '_QC.h5ad'  # post QC (pre-analysis) 
adata = sc.read(resultsFileQC)
adata

In [ ]:
adata.obs.PDX.cat.categories

In [ ]:
#sc.pp.normalize_total(adata,target_sum=1e4)

In [ ]:
#sc.pp.log1p(adata)

In [ ]:
simple = []
for h in adata.obs.Classification:
    spliton = "_"
    if "-" in h:
         spliton = "-"
    simple.append(h.split(spliton)[0])
adata.obs["simpPDX"] = simple
print(np.unique(simple))

In [ ]:
pdxName = "LX55A"
adata = adata[adata.obs.simpPDX==pdxName]

resultsFile = writeDir + fileName + pdxName+'.h5ad'       # final output

In [ ]:
sc.pp.highly_variable_genes(adata, batch_key="PDX")#, flavor = "seurat", n_top_genes=2000)

In [ ]:
minMean = 0.025
maxMean = 2.9
minDisp = 0.7

fig, axs = plt.subplots(1, 2, figsize=(10, 5))

means = adata.var[["means"]][adata.var[["means"]] > np.exp(-14)]#adata.var[["means"]],
axs[0].hist(np.log(means), bins=100)#, log=True),
axs[0].axvline(np.log(minMean), color='k', linestyle='dashed', linewidth=1)
axs[0].axvline(np.log(maxMean), color='k', linestyle='dashed', linewidth=1)
axs[0].set_title('Gene means counts')
axs[0].set_xlabel('means')
axs[0].set_ylabel('counts')

dispNorm = adata.var[["dispersions_norm"]][adata.var[["dispersions_norm"]] > np.exp(-5)]#adata.var[["means"]],
axs[1].hist(np.log(dispNorm), bins=100)#, log=True),
axs[1].axvline(np.log(minDisp), color='k', linestyle='dashed', linewidth=1)
axs[1].set_title('Gene dispersions counts')
axs[1].set_xlabel('dispersions')
axs[1].set_ylabel('counts')

sc.pp.highly_variable_genes(adata, min_disp=minDisp, min_mean=minMean, max_mean=maxMean, batch_key="PDX")
print(sum(adata.var.highly_variable))
print(sum(adata.var.highly_variable_intersection))

In [ ]:
sc.pl.highly_variable_genes(adata)

In [ ]:
#for inGene in adata.var[-8:].index.tolist():
#    adata.var.loc[inGene,"highly_variable"] = False

In [ ]:
adata.var["mt"] = ["MT-" in g for g in adata.var_names]

In [ ]:
adata.raw = adata

In [ ]:
#adata = adata[:, adata.var.highly_variable]
adata = adata[:, np.logical_and(adata.var.highly_variable, np.logical_not(adata.var.mt))]

In [ ]:
#sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])

In [ ]:
#sc.pp.scale(adata, max_value=10)

In [ ]:
sc.tl.pca(adata, n_comps = 100, svd_solver='arpack')

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs = 100, log=True)

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs = 100)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=20, n_pcs=60)

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.tl.leiden(adata, resolution=0.6)

In [ ]:
sc.pl.umap(adata, color=["leiden","PDX", "simpPDX", "Classification","EPCAM","VIM","FBN1","CAPS"],ncols=4)

In [ ]:
sc.pl.pca(adata, color=["leiden","PDX", "simpPDX", "Classification","EPCAM","VIM","FBN1","CAPS"],ncols=4)

In [ ]:
#sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon', use_raw=False)
#sc.tl.filter_rank_genes_groups(adata, groupby="leiden", use_raw=False,
#                                   key_added='rank_genes_groups_filtered', 
#                                   min_in_group_fraction=0.25, min_fold_change=1, max_out_group_fraction=0.5, compare_abs=False)
#sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
#pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(15)

In [ ]:
#sc.tl.dendrogram(adata,groupby="leiden", n_pcs=60)
#sc.pl.rank_genes_groups_dotplot(adata,n_genes=3)#,key="rank_genes_groups_filtered")

In [ ]:
resultsFile

In [ ]:
adata.write(resultsFile)
adata

In [ ]:
homology = pd.read_table("data/hgncHM_121.csv", sep=",")
mouseGenes = np.array(homology["mouse"])
humanGenes = np.array(homology["human"])
homology

In [ ]:
import scanpyHelpers as scH

In [ ]:
adata = sc.read(resultsFile)
adata

In [ ]:
sigFile = "data/LUAD/humanLUADgeneSig.csv"
sigGenes = pd.read_csv(sigFile)
sigGenes

In [ ]:
sigGenes.columns

In [ ]:
scoreNames, newBClabel, ogLabelScoreMed = scH.scoreAndLabel(adata, sigGenes, labelOfSigGenes= sigGenes.columns, ogLabel="leiden")
sc.pl.umap(adata, color=scoreNames+["cellState","leiden"], ncols=4)

In [ ]:
sc.pl.umap(adata, color=scoreNames+["leiden","cellState", 'simpPDX',"PDX"], ncols=4)

In [ ]:
markers = pd.DataFrame(np.empty((500*12,2), dtype="object"), columns=["genes", "clustName"])
for i,clust in enumerate(sigGenes.columns):
    for j,gene in enumerate(sigGenes.index):
        gene = sigGenes.loc[gene,clust]
        indexOver = np.where(mouseGenes == gene)[0]
        if indexOver.size > 0:
            gene = humanGenes[indexOver[0]]
        markers.iloc[i*500+j] = gene,clust
        
markers = markers.dropna()
markers

In [ ]:
import decoupler as dc

In [ ]:
dc.run_ora(
    mat=adata,
    net=markers,
    source='clustName',
    target='genes',
    min_n=3,
    verbose=True
)

In [ ]:
adata.obsm['ora_estimate']


In [ ]:
acts = dc.get_acts(adata, obsm_key='ora_estimate')

# We need to remove inf and set them to the maximum value observed for pvals=0
acts_v = acts.X.ravel()
max_e = np.nanmax(acts_v[np.isfinite(acts_v)])
acts.X[~np.isfinite(acts.X)] = max_e

acts

In [ ]:
sc.pl.umap(acts, color=scoreNames+['leiden'], cmap='RdBu_r')
sc.pl.violin(acts, keys=scoreNames, groupby='leiden')

In [ ]:
adata

In [ ]:
df = dc.rank_sources_groups(acts, groupby='leiden', reference='rest', method='t-test_overestim_var')
df

In [ ]:
n_ctypes = 3
ctypes_dict = df.groupby('group').head(n_ctypes).groupby('group')['names'].apply(lambda x: list(x)).to_dict()
ctypes_dict

In [ ]:
sc.pl.matrixplot(acts, ctypes_dict, 'leiden', dendrogram=True, standard_scale='var',
                 colorbar_title='Z-scaled scores', cmap='RdBu_r')


In [ ]:
annotation_dict = df.groupby('group').head(1).set_index('group')['names'].to_dict()
annotation_dict

In [ ]:
# Add cell type column based on annotation
adata.obs['decoupler'] = [annotation_dict[clust] for clust in adata.obs['leiden']]

# Visualize
sc.pl.umap(adata, color=['decoupler','cellState', "leiden"])

In [ ]:
adata.write(resultsFile)
adata

In [ ]:
sigGenes

In [ ]:
adata = sc.read(resultsFile)
adata.obs_names_make_unique()

In [ ]:
def addIndvLabel(adata, scoreNames, obsLabel="zsig", cutoff=1):
    scoreMat = adata.obs[scoreNames]#[f"{sigName}Score" for sigName in sigGenes.columns]]
    zSig = zScores(scoreMat, cutoff = cutoff)
    adata.obs[obsLabel] = zSig
    return(adata)

def zScores(scoreMat, cutoff = 1.5):
    sigs = []
    for i,scoreCol in enumerate(scoreMat.columns):
        scoreData = scoreMat[scoreCol]
        zscore = ((scoreData-np.mean(scoreData))/np.std(scoreData))
        sigs.append(zscore)
    sigScore = pd.DataFrame(np.array(sigs)).T
    simple = []
    for i,cell in enumerate(sigScore.index):
        names = scoreMat.columns#np.array([str(x+1) for x in range(len(sigScore.columns))])
        
        #print((sigScore.loc[cell]> cutoff).values)
        sigNames = names[np.array((sigScore.loc[cell]> cutoff).values)]
        sigName = "out"
        if len(sigNames) > 0:
            sigName = str(names[np.argmax(sigScore.loc[cell])])
            #sigName = "-".join(sigNames)
        #simple.append("-".join(sigNames))
        simple.append(sigName)
    return(np.array(simple))

In [ ]:
addIndvLabel(adata, [f"{sigName}Score" for sigName in sigGenes.columns], obsLabel="zsig", cutoff=1)

In [ ]:
import pdb
def topPercent(scoreMat, cutoff = .80):
    #perc = pd.DataFrame(np.zeros(scoreMat.shape),columns=scoreMat.columns,index=scoreMat.index)
    #for i,scoreCol in enumerate(scoreMat.columns):
    #    scoreData = scoreMat[scoreCol]
    #    for j, cell in enumerate(scoreData.index):
    #        try:
    #            perc.loc[cell,scoreCol] = sum(scoreData<scoreData[cell])/len(scoreData)
    #        except:
    #            pdb.set_trace()
            #break
        #break
    #print(perc)
    simple=[]
    for i,cell in enumerate(scoreMat.index):
        names = scoreMat.columns
        sigNames = names[np.array((scoreMat.loc[cell]> cutoff).values)]
        sigName = "out"
        if len(sigNames) > 0:
            sigName = str(names[np.argmax(scoreMat.loc[cell])])[:-5]
        simple.append(sigName)
    return(np.array(simple))

In [ ]:
scoreMat = adata.obs[scoreNames]
#adata.obs["topPer"] = topPercent(scoreMat, cutoff=0.8)

In [ ]:
sc.pl.umap(adata, color=["leiden",'decoupler','zsig','cellState'])

In [ ]:
adata.write(resultsFile)
adata